In [12]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets, Output
from IPython.display import display

import plotly.io as pio
pio.templates.default = "plotly_white"

df = pd.read_excel("Student Smoking, Alcohol, and Mental Health Data(1)(1).xlsx")

chart_type = widgets.Dropdown(
    options=['吸烟频率分布', '饮酒频率分布', '心理健康分布', '研究领域对比'],
    value='吸烟频率分布',
    description='图表类型:'
)

button = widgets.Button(
    description='生成图表',
    button_style='success'
)

output = Output()

def generate_chart(b):
    with output:
        output.clear_output()
        chart = chart_type.value

        if chart == '吸烟频率分布':
            smoking_dist = df['吸烟频率'].value_counts()
            fig = px.pie(
                names=smoking_dist.index,
                values=smoking_dist.values,
                title="学生吸烟频率分布",
                hole=0.3
            )
            fig.update_traces(textinfo='percent+label', hoverinfo='label+percent+value')

        elif chart == '饮酒频率分布':
            drinking_dist = df['酒精消费频率'].value_counts().sort_index()
            drinking_df = pd.DataFrame({
                '酒精消费频率': drinking_dist.index,
                '人数': drinking_dist.values
            })
            fig = px.bar(
                drinking_df,
                x='酒精消费频率',
                y='人数',
                text='人数',
                title='学生饮酒频率分布'
            )
            fig.update_traces(textposition='outside')
            fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')

        elif chart == '心理健康分布':
            mental_health = df.groupby(['研究领域', '心理健康']).size().reset_index(name='人数')
            fig = px.bar(
                mental_health,
                x='研究领域',
                y='人数',
                color='心理健康',
                title='不同研究领域心理健康状态分布',
                text='人数'
            )
            fig.update_traces(textposition='inside')
            fig.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})

        elif chart == '研究领域对比':
            smoking_freq = df.groupby(['研究领域', '吸烟频率']).size().unstack(fill_value=0)
            drinking_freq = df.groupby(['研究领域', '酒精消费频率']).size().unstack(fill_value=0)

            fig = go.Figure()

            for freq in smoking_freq.columns:
                fig.add_trace(go.Scatter(
                    x=smoking_freq.index,
                    y=smoking_freq[freq],
                    mode='lines+markers',
                    name=f'吸烟: {freq}',
                    marker=dict(symbol='circle')
                ))

            for freq in drinking_freq.columns:
                fig.add_trace(go.Scatter(
                    x=drinking_freq.index,
                    y=drinking_freq[freq],
                    mode='lines+markers',
                    name=f'饮酒: {freq}',
                    yaxis='y2',
                    marker=dict(symbol='square')
                ))

            fig.update_layout(
                title="研究领域吸烟与饮酒频率对比",
                xaxis_title="研究领域",
                yaxis=dict(
                    title=dict(text='吸烟人数', font=dict(color='blue')),
                    tickfont=dict(color='blue')
                ),
                yaxis2=dict(
                    title=dict(text='饮酒人数', font=dict(color='red')),
                    tickfont=dict(color='red'),
                    overlaying='y',
                    side='right'
                ),
                legend=dict(x=0.01, y=0.99),
                hovermode='x unified'
            )

        fig.show()

button.on_click(generate_chart)

display(widgets.VBox([chart_type, button, output]))